In [1]:
from egg.zoo.vd_reco import data_proto
dataset = data_proto.Data(fn="../protoroles_eng_pb/protoroles_eng_pb_08302015.tsv",
                          augment="basic")
idx2role = {idx: role for role, idx in dataset.idx_roleset.items()}

total sentence, tokens and arguments: 9270
total sentences and verb tokens: 5193
args count Counter({1: 4163, 0: 3939, 2: 1168})
removing 48 examples which have no roles (n_max_args=3)
Counter({(True, True, False): 2390, (True, False, False): 931, (False, True, False): 760, (False, True, True): 456, (True, True, True): 443, (True, False, True): 151, (False, False, False): 48, (False, False, True): 14})


In [2]:
import numpy as np

args = []
for d in dataset:
    sender_inputs, _, _ = d
    _, properties, to_send = sender_inputs
    for j, v in enumerate(to_send[1:]):
        if v == 1:
            args.append(properties[j])
args = np.stack(args)

# PCA
As a first, very crude approximation, we can try to apply PCA on the data. It is very crude because data is not continuous, and moreover, there are lots of N/A. Still, let's see...


In [3]:
# preprocess these arguments
args_cont = args.copy()
print(args_cont[:3])
args_cont[args_cont == 0.] = 3
print(args_cont[:3])
args_cont = (args_cont - 2)
print(args_cont[:3])


[[3 0 2 0 1 1 3 3 3 0 3 0 0 0 2 0 0 3]
 [0 0 2 0 3 1 3 1 3 0 0 0 0 3 3 0 0 0]
 [0 0 3 0 1 1 3 3 3 0 0 0 0 0 3 0 0 0]]
[[3 3 2 3 1 1 3 3 3 3 3 3 3 3 2 3 3 3]
 [3 3 2 3 3 1 3 1 3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3 3]]
[[ 1  1  0  1 -1 -1  1  1  1  1  1  1  1  1  0  1  1  1]
 [ 1  1  0  1  1 -1  1 -1  1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1]]


In [4]:
from sklearn.decomposition import PCA

model = PCA(n_components=2, whiten=True)
model.fit(args_cont)

PCA(n_components=2, whiten=True)

In [5]:
model.components_

array([[-0.02944897, -0.21898863, -0.24468966, -0.09221056, -0.57540033,
        -0.33973404,  0.03825623,  0.27456393,  0.03005701, -0.00979724,
        -0.06283318, -0.01202883, -0.18607906, -0.00391557, -0.48500489,
        -0.02530752, -0.2792066 , -0.0767325 ],
       [-0.03376203, -0.13596905, -0.37833404,  0.13888911,  0.53367329,
         0.14267766,  0.04487401, -0.34723074,  0.00254301, -0.05949469,
         0.05476461,  0.00941763, -0.1387911 , -0.01081923, -0.59541331,
        -0.01545405, -0.09683828,  0.0071624 ]])

In [6]:
comp_prop0 = sorted(zip(model.components_[0], dataset.ordered_properties))
comp_prop1 = sorted(zip(model.components_[1], dataset.ordered_properties))

In [7]:
comp_prop0

[(-0.5754003315347728, 'created'),
 (-0.48500488560631866, 'predicate_changed_argument'),
 (-0.339734036705558, 'destroyed'),
 (-0.2792066008145787, 'stationary'),
 (-0.24468966081250945, 'change_of_state'),
 (-0.21898863461130985, 'change_of_location'),
 (-0.1860790608705551, 'makes_physical_contact'),
 (-0.09221055793573503, 'changes_possession'),
 (-0.0767324953793391, 'volition'),
 (-0.06283317818091252, 'instigation'),
 (-0.0294489698031192, 'awareness'),
 (-0.025307515688507788, 'sentient'),
 (-0.012028831184816314, 'location_of_event'),
 (-0.00979724380079415, 'exists_as_physical'),
 (-0.003915565454983711, 'manipulated_by_another'),
 (0.03005701439624688, 'existed_during'),
 (0.03825623290890032, 'existed_after'),
 (0.27456392970781734, 'existed_before')]

In [8]:
comp_prop1

[(-0.5954133129103042, 'predicate_changed_argument'),
 (-0.37833403966906065, 'change_of_state'),
 (-0.3472307351544367, 'existed_before'),
 (-0.13879110122853747, 'makes_physical_contact'),
 (-0.1359690510152945, 'change_of_location'),
 (-0.09683827813875047, 'stationary'),
 (-0.05949469300048274, 'exists_as_physical'),
 (-0.0337620273640992, 'awareness'),
 (-0.015454049624351008, 'sentient'),
 (-0.010819226745428694, 'manipulated_by_another'),
 (0.002543005258685132, 'existed_during'),
 (0.007162397693808182, 'volition'),
 (0.009417630719648058, 'location_of_event'),
 (0.044874007566452996, 'existed_after'),
 (0.05476460591882306, 'instigation'),
 (0.1388891137391592, 'changes_possession'),
 (0.14267765891394693, 'destroyed'),
 (0.5336732889137092, 'created')]

# KMedoids

from https://faculty.ucmerced.edu/mcarreira-perpinan/papers/lapkmodes-slides.pdf :

> K-medoids: exemplars are generally valid but may be noisy or atypical.

The K-mode algorithm seems interesting for that purpose.

In [9]:
from sklearn_extra.cluster import KMedoids

kmedoids = KMedoids(n_clusters=2, metric='hamming', init='k-medoids++').fit(args)
comp_prop0 = sorted(zip(kmedoids.cluster_centers_[0], dataset.ordered_properties))
comp_prop1 = sorted(zip(kmedoids.cluster_centers_[1], dataset.ordered_properties))
print(comp_prop0)
print(comp_prop1)

[(0, 'awareness'), (0, 'change_of_location'), (0, 'changes_possession'), (0, 'exists_as_physical'), (0, 'instigation'), (0, 'location_of_event'), (0, 'makes_physical_contact'), (0, 'sentient'), (0, 'stationary'), (0, 'volition'), (1, 'created'), (1, 'destroyed'), (3, 'change_of_state'), (3, 'existed_after'), (3, 'existed_before'), (3, 'existed_during'), (3, 'manipulated_by_another'), (3, 'predicate_changed_argument')]
[(0, 'changes_possession'), (0, 'location_of_event'), (0, 'manipulated_by_another'), (1, 'created'), (1, 'destroyed'), (1, 'predicate_changed_argument'), (2, 'change_of_location'), (2, 'change_of_state'), (2, 'makes_physical_contact'), (2, 'stationary'), (3, 'awareness'), (3, 'existed_after'), (3, 'existed_before'), (3, 'existed_during'), (3, 'exists_as_physical'), (3, 'instigation'), (3, 'sentient'), (3, 'volition')]


In [10]:
from sklearn_extra.cluster import KMedoids

# method='pam' is supposed to be more accurate
# kmedoids = KMedoids(n_clusters=2, metric='hamming', init='k-medoids++', method='pam').fit(args)
comp_prop0 = sorted(zip(kmedoids.cluster_centers_[0], dataset.ordered_properties))
comp_prop1 = sorted(zip(kmedoids.cluster_centers_[1], dataset.ordered_properties))
print(comp_prop0)
print(comp_prop1)

[(0, 'awareness'), (0, 'change_of_location'), (0, 'changes_possession'), (0, 'exists_as_physical'), (0, 'instigation'), (0, 'location_of_event'), (0, 'makes_physical_contact'), (0, 'sentient'), (0, 'stationary'), (0, 'volition'), (1, 'created'), (1, 'destroyed'), (3, 'change_of_state'), (3, 'existed_after'), (3, 'existed_before'), (3, 'existed_during'), (3, 'manipulated_by_another'), (3, 'predicate_changed_argument')]
[(0, 'changes_possession'), (0, 'location_of_event'), (0, 'manipulated_by_another'), (1, 'created'), (1, 'destroyed'), (1, 'predicate_changed_argument'), (2, 'change_of_location'), (2, 'change_of_state'), (2, 'makes_physical_contact'), (2, 'stationary'), (3, 'awareness'), (3, 'existed_after'), (3, 'existed_before'), (3, 'existed_during'), (3, 'exists_as_physical'), (3, 'instigation'), (3, 'sentient'), (3, 'volition')]


## Number of clusters

What happens when we use more clusters?

In [11]:
from sklearn_extra.cluster import KMedoids

n=6

kmedoids = KMedoids(n_clusters=n, metric='hamming', init='k-medoids++').fit(args)
for i in range(0,n):
    comp_prop = sorted(zip(kmedoids.cluster_centers_[i], dataset.ordered_properties))
    print(i,)
    for V, attr in comp_prop: 
        if V != 0 and V != 2:
            print(attr, V,)

0
created 1
destroyed 1
instigation 1
volition 1
awareness 3
change_of_state 3
existed_after 3
existed_before 3
existed_during 3
exists_as_physical 3
manipulated_by_another 3
predicate_changed_argument 3
sentient 3
1
created 1
destroyed 1
existed_after 3
existed_before 3
existed_during 3
manipulated_by_another 3
2
created 1
destroyed 1
predicate_changed_argument 1
awareness 3
existed_after 3
existed_before 3
existed_during 3
exists_as_physical 3
instigation 3
sentient 3
volition 3
3
created 1
destroyed 1
change_of_state 3
existed_after 3
existed_before 3
existed_during 3
manipulated_by_another 3
predicate_changed_argument 3
4
manipulated_by_another 3
5
destroyed 1
existed_before 1
change_of_state 3
created 3
existed_after 3
existed_during 3
manipulated_by_another 3
predicate_changed_argument 3


The results of White are slightly different. In his "computational linking theory" paper, there is one proto-agent and several proto-patients. Here we find 2 agents with volition and instigation.

# Prototypical structure of verb types

Can we retrieve classical roles without looking at syntax? In other words, are classical roles simply proto-types in the semantic space?

## Naive

In the first part of the notebook, we test whether we can retrieve classical roles by clustering instances of arguments. It turns out we can. 

Aaron White's computational linking model is a bit different, because the prototype founds must explain the linguistic data. Here, there is no linguistic data at all.

The naive version does:

* First, run a k-medoid clustering algorithm on all the data. This algorithm works on categorical data, unlike k-means.
* For each role of each verb, create histogram of arguments by taking the closest prototype.
* Run k-means on these histograms to group them.
    * __Problem__: So far, there is no constraint that the clustering respect the fact that different arguments of the same verb should belong to different clusters.

In [12]:
argument_prototypes = kmedoids.cluster_centers_
argument_prototypes

array([[3, 2, 3, 0, 1, 1, 3, 3, 3, 3, 1, 0, 2, 3, 3, 3, 2, 1],
       [0, 0, 2, 0, 1, 1, 3, 3, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [3, 2, 2, 0, 1, 1, 3, 3, 3, 3, 3, 0, 2, 0, 1, 3, 2, 3],
       [0, 0, 3, 0, 1, 1, 3, 3, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 3, 0, 3, 1, 3, 1, 3, 0, 0, 0, 0, 3, 3, 0, 0, 0]])

In [26]:
from collections import defaultdict, Counter

n_verb_types = Counter()
# first, count verb types
# this is useful because right now we discard verbs on which we don't have enough data
for d in dataset:
    sender_inputs, _, _ = d
    roleset, _, _ = sender_inputs
    n_verb_types[roleset] += 1
# print(n_verb_types)

# for each verb type, find the closest
frequent_args = defaultdict(lambda: defaultdict(list))
for d in dataset:
    sender_inputs, _, _ = d
    roleset, properties, to_send = sender_inputs
    if n_verb_types[roleset] < 2:
        continue
#     else:
#         print("H")
    for j, v in enumerate(to_send[1:]):
        if v == 1:
            # compute hamming distance and select the prototype with the least distance
            diff = (properties[j] != argument_prototypes)
            dist = diff.astype(int).sum(1)
            frequent_args[roleset][j].append(dist.argmin())

At this point for each roleset and each argument j, a list of cluster assignments.

In [27]:
# then we can compute a distribution for each by normalizing,
args_distrib = defaultdict(dict)
for roleset, A in frequent_args.items():
    for arg_id, matched in A.items():
        distrib = np.zeros(len(argument_prototypes), dtype=float)
        np.add.at(distrib, np.asarray(matched), 1)
        distrib /= len(matched)
        args_distrib[roleset][arg_id] = distrib

In [67]:
# find clusters
from sklearn.cluster import KMeans
all_distribs = []
for roleset, distribs in args_distrib.items():
    all_distribs.extend(distribs.values())
all_distribs = np.vstack(all_distribs)
for K in range(80, 80):
    model = KMeans(n_clusters=K).fit(all_distribs)
    score = model.score(all_distribs)
    BIC = -2 * score + np.log([len(all_distribs)])[0] * K
    print(K, BIC)
    arg_prototypes = model.cluster_centers_

In [68]:
arg_prototypes

array([[ 2.08166817e-16,  6.10622664e-16,  9.99881587e-01,
        -9.99200722e-16,  1.18413262e-04,  4.16333634e-17],
       [ 8.08472795e-05,  9.99593143e-01, -1.05471187e-15,
        -7.77156117e-16,  1.86364869e-04,  1.39645301e-04],
       [ 3.45781466e-04,  0.00000000e+00, -9.43689571e-16,
         9.99654219e-01,  1.66533454e-16, -4.16333634e-17],
       [ 6.93889390e-18,  5.55111512e-17,  5.55111512e-17,
         2.88888889e-01,  7.11111111e-01,  1.38777878e-17],
       [ 5.55111512e-17, -5.55111512e-17,  2.22044605e-16,
         1.94289029e-16, -1.11022302e-16,  1.00000000e+00],
       [ 9.96321429e-01, -2.22044605e-16,  1.42857143e-03,
         1.00000000e-03, -2.77555756e-17,  1.25000000e-03],
       [-6.93889390e-18,  3.21840986e-01,  6.78159014e-01,
         0.00000000e+00,  2.77555756e-17,  2.77555756e-17],
       [ 2.08256547e-01,  5.05685619e-01,  3.69121562e-02,
         2.04927991e-01,  2.04081633e-02,  2.38095238e-02],
       [ 5.55111512e-17,  1.28266795e-03,  8.417

In [69]:
classical_role_binning = defaultdict(Counter)
for roleset, distribs in args_distrib.items():
    for arg_id, distrib in distribs.items():
        L2_to_prototypes = ((distrib-arg_prototypes)**2).sum(1)
        assigned = L2_to_prototypes.argmin()
        classical_role_binning[assigned][arg_id] += 1

for id_, classical_roles in classical_role_binning.items():
    norm_roles = np.asarray(list(classical_roles.values()), dtype=float)
    norm_roles /= norm_roles.sum()
    print(id_, classical_roles, sorted(norm_roles, reverse=True))

49 Counter({0: 13, 2: 3, 1: 2}) [0.7222222222222222, 0.16666666666666666, 0.1111111111111111]
40 Counter({1: 8, 2: 3}) [0.7272727272727273, 0.2727272727272727]
48 Counter({1: 10, 2: 1}) [0.9090909090909091, 0.09090909090909091]
0 Counter({0: 477, 1: 65, 2: 21}) [0.8472468916518651, 0.11545293072824156, 0.037300177619893425]
36 Counter({1: 13, 0: 2, 2: 1}) [0.8125, 0.125, 0.0625]
1 Counter({1: 309, 0: 214, 2: 128}) [0.47465437788018433, 0.32872503840245776, 0.1966205837173579]
2 Counter({1: 350, 0: 76, 2: 56}) [0.7261410788381742, 0.15767634854771784, 0.11618257261410789]
84 Counter({0: 19}) [1.0]
10 Counter({1: 13, 0: 4, 2: 4}) [0.6190476190476191, 0.19047619047619047, 0.19047619047619047]
34 Counter({0: 5, 1: 3, 2: 1}) [0.5555555555555556, 0.3333333333333333, 0.1111111111111111]
15 Counter({0: 31, 1: 12, 2: 2}) [0.6888888888888889, 0.26666666666666666, 0.044444444444444446]
21 Counter({1: 28, 0: 11, 2: 6}) [0.6222222222222222, 0.24444444444444444, 0.13333333333333333]
87 Counter({0: 8

In [70]:
# let's compute purity
n_points = 0
correct_points = 0
for id_, classical_roles in classical_role_binning.items():
    id_most_common, val_most_common = classical_roles.most_common(1)[0]
    n_points += sum(classical_roles.values())
    correct_points += val_most_common
print(correct_points/n_points)

0.6832219038522763


## Results

We can see that some bins closely correspond to classical roles: bin 0, 6 and 2 are very agent-like, 5, 7, 1 are more patient like and some like 3 and 4 correspond to other roles more.

## Implementing constraint

Once we have k-means clusters, what happens if we constrain the different argument histograms of the same verb to have different assignments? Does it improve purity?

In [71]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import min_weight_full_bipartite_matching

classical_role_binning = defaultdict(Counter)
for roleset, distribs in args_distrib.items():
    distances = []
    assigned_no_constraints = []
    for arg_id, distrib in distribs.items():
        d = ((distrib-arg_prototypes)**2).sum(1)
        assigned_no_constraints.append(d.argmin())
        distances.append(d)
    distances = csr_matrix(np.asarray(distances))
    assign = min_weight_full_bipartite_matching(distances)
    #print(assign[1], assigned_no_constraints)
    for arg_id, k in zip(distribs.keys(), assign[1]):
        classical_role_binning[k][arg_id] += 1

    #classical_role_binning[assigned][arg_id] += 1

for id_, classical_roles in classical_role_binning.items():
    norm_roles = np.asarray(list(classical_roles.values()), dtype=float)
    norm_roles /= norm_roles.sum()
    print(id_, classical_roles, sorted(norm_roles, reverse=True))

49 Counter({0: 13, 2: 9, 1: 2}) [0.5416666666666666, 0.375, 0.08333333333333333]
40 Counter({1: 8, 2: 3}) [0.7272727272727273, 0.2727272727272727]
48 Counter({1: 10, 2: 1}) [0.9090909090909091, 0.09090909090909091]
0 Counter({0: 477, 1: 40, 2: 12}) [0.9017013232514177, 0.07561436672967864, 0.022684310018903593]
36 Counter({1: 14, 0: 2, 2: 1}) [0.8235294117647058, 0.11764705882352941, 0.058823529411764705]
1 Counter({1: 253, 0: 214, 2: 80}) [0.4625228519195612, 0.3912248628884826, 0.14625228519195613]
2 Counter({1: 310, 0: 75, 2: 34}) [0.7398568019093079, 0.17899761336515513, 0.081145584725537]
84 Counter({0: 20, 1: 1}) [0.9523809523809523, 0.047619047619047616]
10 Counter({1: 15, 0: 4, 2: 4}) [0.6521739130434783, 0.17391304347826086, 0.17391304347826086]
34 Counter({0: 5, 1: 3, 2: 1}) [0.5555555555555556, 0.3333333333333333, 0.1111111111111111]
15 Counter({0: 31, 1: 12, 2: 2}) [0.6888888888888889, 0.26666666666666666, 0.044444444444444446]
21 Counter({1: 27, 0: 11, 2: 5}) [0.6279069767

In [72]:
# let's compute purity
n_points = 0
correct_points = 0
for id_, classical_roles in classical_role_binning.items():
    id_most_common, val_most_common = classical_roles.most_common(1)[0]
    n_points += sum(classical_roles.values())
    correct_points += val_most_common
print(correct_points/n_points)

0.6860872333651703


### Model

Limitations

- Neglects the ordinal + N/A nature of the original data
- Breakdown of the problem into problems to be solved greedily

Possible improvements

- Use BIC or AIC to find K?


If I ever use learned prototypes instead of classical roles, I will have the following problem:

- Depending on the value of the "second" K value of the K-means, the input repr will be very different. If K=2, the strategy for which the network encodes the position will work (as it is working right now with K=3 and the classical roles). But if K=10 or K=50, which yields a larger purity (but barely)!, then this strategy might break. What would be learned, then? 

Alternative model 1: *probabilistic mixture model*

- Compare models via likelihood
- Now a prototype is not a vector of the dataset anymore, but a probability distribution, and same for cluster attribution. 

Alternative model 2: *graphVAE*

- 

### Data

Limits of the data: small data: 10 instances of verbs MAX might be too small?




    